In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/LLMS_LLAVA

/content/gdrive/MyDrive/LLMS_LLAVA


In [3]:
!pip install torch transformers datasets pillow
!pip install --upgrade datasets pyarrow pandas

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, CLIPModel, CLIPProcessor, AutoProcessor
from datasets import load_dataset, load_from_disk
from PIL import Image
from io import BytesIO
import requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 52.2 MB/s eta 0:00:00
  

In [ ]:
# !wget 'https://huggingface.co/datasets/liuhaotian/LLaVA-CC3M-Pretrain-595K/resolve/main/images.zip'
!unzip images.zip -d images

Streaming output truncated to the last 5000 lines.
  inflating: images/GCC_train_001135613.jpg  
  inflating: images/GCC_train_001670720.jpg  
  inflating: images/GCC_train_000809415.jpg  
  inflating: images/GCC_train_000591249.jpg  
  inflating: images/GCC_train_002059915.jpg  
  inflating: images/GCC_train_001864158.jpg  
  inflating: images/GCC_train_002290652.jpg  
  inflating: images/GCC_train_002301886.jpg  
  inflating: images/GCC_train_002007278.jpg  
  inflating: images/GCC_train_000014614.jpg  
  inflating: images/GCC_train_002259006.jpg  
  inflating: images/GCC_train_000552698.jpg  
  inflating: images/GCC_train_000246645.jpg  
  inflating: images/GCC_train_000488140.jpg  
  inflating: images/GCC_train_002422257.jpg  
  inflating: images/GCC_train_000405594.jpg  
  inflating: images/GCC_train_001712750.jpg  
  inflating: images/GCC_train_001713324.jpg  
  inflating: images/GCC_train_000685482.jpg  
  inflating: images/GCC_train_001004315.jpg  
  inflating: images/GCC_train

In [ ]:
!ls images


Streaming output truncated to the last 5000 lines.
GCC_train_000385808.jpg  GCC_train_001139551.jpg  GCC_train_002071367.jpg  GCC_train_002590185.jpg
GCC_train_000385809.jpg  GCC_train_001139561.jpg  GCC_train_002071369.jpg  GCC_train_002590186.jpg
GCC_train_000385812.jpg  GCC_train_001139566.jpg  GCC_train_002071370.jpg  GCC_train_002590188.jpg
GCC_train_000385857.jpg  GCC_train_001139574.jpg  GCC_train_002071374.jpg  GCC_train_002590193.jpg
GCC_train_000385858.jpg  GCC_train_001139587.jpg  GCC_train_002071377.jpg  GCC_train_002590194.jpg
GCC_train_000385869.jpg  GCC_train_001139591.jpg  GCC_train_002071379.jpg  GCC_train_002590196.jpg
GCC_train_000385870.jpg  GCC_train_001139598.jpg  GCC_train_002071384.jpg  GCC_train_002590202.jpg
GCC_train_000385874.jpg  GCC_train_001139603.jpg  GCC_train_002071392.jpg  GCC_train_002590206.jpg
GCC_train_000385883.jpg  GCC_train_001139615.jpg  GCC_train_002071397.jpg  GCC_train_002590210.jpg
GCC_train_000385887.jpg  GCC_train_001139619.jpg  GCC_trai

In [4]:
dataset = load_dataset('liuhaotian/LLaVA-CC3M-Pretrain-595K', data_files='chat.json')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

chat.json:   0%|          | 0.00/211M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
metadata = load_dataset('liuhaotian/LLaVA-CC3M-Pretrain-595K', data_files='metadata.json')

metadata.json:   0%|          | 0.00/199M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset['train'][1000]

{'id': 'GCC_train_002605796',
 'image': 'GCC_train_002605796.jpg',
 'conversations': [{'from': 'human',
   'value': 'Give a short and clear explanation of the subsequent image.\n<image>'},
  {'from': 'gpt',
   'value': 'building is getting a lot of interest lately by developers looking to open a bar or restaurant .'}]}

In [5]:
import os

def add_image_path(example):
    image_filename = example['image']

    # Construct the full path to the image file
    example['image'] = os.path.join('images', image_filename)
    return example

In [6]:
dataset = dataset.map(add_image_path)


Map:   0%|          | 0/595375 [00:00<?, ? examples/s]

In [ ]:
metadata['train'][0]

{'id': 'GCC_train_002582585',
 'image': 'GCC_train_002582585.jpg',
 'caption': 'olive oil is a healthy ingredient used liberally .',
 'blip_caption': 'some olive oil and salt in a glass bottle',
 'url': 'https://i.pinimg.com/736x/de/13/3a/de133a155c777a9db265bb3e7888719d--colombo-olive-oils.jpg'}

In [7]:
!pip install safetensors

# Load the Vicuna model
model_name = "lmsys/vicuna-7b-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
vicuna_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="offload"
)


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [8]:
from transformers import CLIPVisionModel, CLIPProcessor

# Load the CLIP model and processor
clip_model_name = "openai/clip-vit-large-patch14"
clip_model = CLIPVisionModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [9]:
vicuna_model.train()
# Add <image> token as a special token
special_tokens_dict = {"additional_special_tokens": ["<image>", "###"]}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)
if num_added_tokens > 0:
  with torch.no_grad():
    vicuna_model.resize_token_embeddings(len(tokenizer))
    vicuna_model.config.vocab_size = len(tokenizer)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [10]:
print(f"Tokenizer vocab size: {len(tokenizer)}")
print(f"Embedding size: {vicuna_model.get_input_embeddings().weight.size(0)}")
print(f"Config vocab size: {vicuna_model.config.vocab_size}")

Tokenizer vocab size: 32002
Embedding size: 32002
Config vocab size: 32002


In [11]:
class VisualProjection(nn.Module):
    def __init__(self, vision_dim, embed_dim):
        super(VisualProjection, self).__init__()
        self.proj = nn.Linear(vision_dim, embed_dim)

    def forward(self, x):
        return self.proj(x)


In [12]:
class MultimodalLLM(nn.Module):
    def __init__(self, vision_model, visual_projection, llm_model):
        super(MultimodalLLM, self).__init__()
        self.vision_model = vision_model
        self.visual_projection = visual_projection
        self.llm_model = llm_model

    def forward(self, input_ids, attention_mask=None, images=None, labels=None):
        if images is not None:
            # Process images through the vision encoder
            vision_outputs = self.vision_model(pixel_values=images, output_hidden_states=True)
            # Extract features from the specified layer
            vision_hidden_states = vision_outputs.hidden_states[select_layer]
            # Use mean pooling over spatial dimensions (excluding CLS token)
            vision_features = vision_hidden_states[:, 1:, :].mean(dim=1)
            # Project visual features to the LLM embedding space
            projected_features = self.visual_projection(vision_features)
            # Get text embeddings
            text_embeddings = self.llm_model.get_input_embeddings()(input_ids)
            # Replace the <image> token embedding with the projected visual features
            image_token_id = tokenizer.convert_tokens_to_ids("<image>")
            inputs_embeds = text_embeddings.clone()
            for i in range(inputs_embeds.size(0)):  # Batch size
                image_positions = (input_ids[i] == image_token_id).nonzero(as_tuple=True)[0]
                if len(image_positions) > 0:
                    pos = image_positions[0]
                    inputs_embeds[i, pos, :] = projected_features[i]
            # Adjust labels to ignore the <image> token during loss computation
            if labels is not None:
                labels = labels.clone()
                labels[input_ids == image_token_id] = -100
            # Pass through the LLM
            outputs = self.llm_model(
                inputs_embeds=inputs_embeds,
                attention_mask=attention_mask,
                labels=labels,
            )
        else:
            # Standard LLM forward pass
            outputs = self.llm_model(
                input_ids=input_ids, attention_mask=attention_mask, labels=labels
            )
        return outputs


In [13]:
# Get dimensions
select_layer = -2  # Second-to-last layer
vision_dim = clip_model.config.hidden_size
embed_dim = vicuna_model.config.hidden_size

# Initialize the projection layer
visual_projection = VisualProjection(vision_dim, embed_dim)


In [ ]:
model = MultimodalLLM(clip_model, visual_projection, vicuna_model)

Tokenizer vocab size: 32002
LLM Model vocab size: 32002
LLM Model vocab size: 32002


In [15]:
# 5. Freeze the vision encoder and LLM parameters
for param in model.vision_model.parameters():
    param.requires_grad = False

for param in model.llm_model.parameters():
    param.requires_grad = False

# Ensure that the projection layer is trainable
for param in model.visual_projection.parameters():
    param.requires_grad = True

Preprocess Conversations

In [16]:
def process_conversations(input_data, add_generation_prompt=False):
    def raise_exception(msg):
        raise ValueError(msg)

    # Initialize system message and output
    system_message = "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions."
    stop_token = '</s>'
    output = '<s>' + system_message + '\n\n'

    # Process each message in the conversations
    for index, message in enumerate(input_data):
        sender = message.get('from')
        content = message.get('value', '').strip()

        if not sender or not content:
            raise_exception("Each conversation entry must have 'from' and 'value' keys.")

        is_human = sender == 'human'
        expected_human_role = (index % 2 == 0)

        # Check for alternating roles
        if is_human != expected_human_role:
            raise_exception('Conversation roles must alternate human/gpt/human/gpt/...')

        # Format the message
        if is_human:
            output += f" HUMAN: {content}\n"
        elif sender == 'gpt':
            output += f" ASSISTANT: {content}{stop_token}\n"

    # Optionally add the generation prompt
    if add_generation_prompt:
        output += "ASSISTANT:"

    return output

In [17]:
dataset.save_to_disk(os.getcwd())

Saving the dataset (0/1 shards):   0%|          | 0/595375 [00:00<?, ? examples/s]

In [ ]:
# Working - needs to be double checked

class CC3MDataset(Dataset):
    def __init__(self, data_path, image_folder, tokenizer, vision_processor):
        # Load the data
        # with open(data_path, ), 'r') as f:
        data = load_from_disk(data_path)

        self.data = data
        self.image_folder = image_folder
        self.tokenizer = tokenizer
        self.vision_processor = vision_processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        conversation = sample['conversations']
        image_id = sample['image']

        # Process the image
        image_path = os.path.join(self.image_folder, image_id)
        image = Image.open(image_path).convert("RGB")
        image_tensor = self.vision_processor(images=image, return_tensors="pt").pixel_values[0]

        # Process the conversation
        input_text = self._build_input_text(conversation)
        encoding = self.tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
            padding='max_length',
            padding_side='right'
        )
        input_ids = encoding['input_ids'][0]
        attention_mask = encoding['attention_mask'][0]

        # Labels (copy of input_ids)
        labels = input_ids.clone()
        # Set labels corresponding to the <image> token to -100
        image_token_id = self.tokenizer.convert_tokens_to_ids("<image>")
        labels[input_ids == image_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "images": image_tensor,
            "labels": labels
        }

    def _build_input_text(self, conversation):
        # Concatenate the conversation turns

        return process_conversations(conversation)


In [19]:
#TODO
## DON"T KNOW IF RIGHT

# Probably still not working - did not update / make edits

class CC3MDataset(Dataset):
    def __init__(self, data_path, image_folder, tokenizer, vision_processor):
        # Load the data
        with open(data_path, 'r') as f:
            data = json.load(f)

        self.data = data
        self.image_folder = image_folder
        self.tokenizer = tokenizer
        self.vision_processor = vision_processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        conversation = sample['conversations']
        image_id = sample['image']

        # Process the image
        image_path = os.path.join(self.image_folder, image_id)
        image = Image.open(image_path).convert("RGB")
        image_tensor = self.vision_processor(images=image, return_tensors="pt").pixel_values[0]

        # Process the conversation
        input_text = self._build_input_text(conversation)
        encoding = self.tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
            padding='max_length',
            padding_side='right'
        )
        input_ids = encoding['input_ids'][0]
        attention_mask = encoding['attention_mask'][0]

        # Create labels
        labels = input_ids.clone()
        # Mask out tokens not part of the assistant's response
        labels = self.mask_labels(input_ids, labels)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "images": image_tensor,
            "labels": labels
        }

    def _build_input_text(self, conversation):
        # System message
        system_message = "You are a helpful assistant."
        stop_token = "###"
        text = f"{system_message} {stop_token}\n"

        for idx in range(0, len(conversation), 2):
            # Human turn
            human_turn = conversation[idx]['value']
            if idx == 0:
                # First turn: randomly place image before or after the question
                image_token = "<image>"
                if self.random_choice():
                    instruction = f"{human_turn}\n{image_token}"
                else:
                    instruction = f"{image_token}\n{human_turn}"
                text += f"Human: {instruction} {stop_token}\n"
            else:
                text += f"Human: {human_turn} {stop_token}\n"

            # Assistant turn
            if idx + 1 < len(conversation):
                assistant_response = conversation[idx + 1]['value']
                text += f"Assistant: {assistant_response} {stop_token}\n"

        return text.strip()

    def random_choice(self):
        return torch.rand(1).item() > 0.5

    def mask_labels(self, input_ids, labels):
        # Convert special tokens to IDs
        stop_token_id = self.tokenizer.convert_tokens_to_ids("###")
        assistant_token_id = self.tokenizer.convert_tokens_to_ids("Assistant:")
        human_token_id = self.tokenizer.convert_tokens_to_ids("Human:")
        image_token_id = self.tokenizer.convert_tokens_to_ids("<image>")

        # Initialize labels to IGNORE_INDEX
        labels[:] = -100

        # Find positions of "Assistant:" tokens
        assistant_token_ids = assistant_token_id if isinstance(assistant_token_id, int) else assistant_token_id[0]
        assistant_positions = (input_ids == assistant_token_ids).nonzero(as_tuple=True)[0]

        for pos in assistant_positions:
            # Start position is after "Assistant:" token
            start = pos + 1
            # Find the next stop token
            end_positions = (input_ids == stop_token_id).nonzero(as_tuple=True)[0]
            end_positions = end_positions[end_positions > start]
            end = end_positions[0] if len(end_positions) > 0 else len(input_ids)
            # Unmask the assistant's response tokens
            labels[start:end] = input_ids[start:end]

        return labels

In [22]:
path = ''
# 7. Create the DataLoader
dataset = CC3MDataset(
    data_path=os.path.join(path, 'train'),
    image_folder=path,
    tokenizer=tokenizer,
    vision_processor=clip_processor
)


In [23]:
# Collate function to handle variable-length sequences
def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    images = torch.stack([item['images'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'images': images,
        'labels': labels
    }

In [24]:

dataloader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=4
)

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to_empty(device=device)
num_epochs = 1
learning_rate = 2e-3
gradient_accumulation_steps = 1  # can be faster if it's larger?
weight_decay = 0.0

# Optimizer
optimizer = torch.optim.AdamW(
    model.visual_projection.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay
)

# Mixed Precision
scaler = torch.GradScaler()
model.train()
optimizer.zero_grad()


In [ ]:
%pwd

'/content/gdrive/MyDrive/LLMS_LLAVA'

In [ ]:
import os
import zipfile

# Path to the zip file and extraction directory
zip_file_path = "images.zip"
extraction_dir = "images"

# Supported image file extensions
image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp')

# Check images in the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_images = [
        file for file in zip_ref.namelist()
        if file.lower().endswith(image_extensions)
    ]

# Check images in the extracted directory
extracted_images = [
    file for file in os.listdir(extraction_dir)
    if file.lower().endswith(image_extensions)
]

# Compare counts
print(f"Images in zip file: {len(zip_images)}")
print(f"Images in extracted directory: {len(extracted_images)}")

if len(zip_images) == len(extracted_images):
    print("All images were successfully extracted.")
else:
    print("Some images might be missing. Please verify.")

Images in zip file: 595375
Images in extracted directory: 595375
All images were successfully extracted.


In [ ]:
from torch.cuda.amp import autocast
image_token_id = tokenizer.convert_tokens_to_ids("<image>")


for epoch in range(num_epochs):
    for step, batch in enumerate(dataloader):
        print(step)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        images = batch["images"].to(device)
        labels = batch["labels"].to(device)

        with autocast():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                images=images,
                labels=labels,
            )
            loss = outputs.loss / gradient_accumulation_steps

        scaler.scale(loss).backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            print(f"Epoch {epoch + 1}, Step {step + 1}, Loss: {loss.item() * gradient_accumulation_steps}")

print("Training completed.")


<image> token ID: 32000
Tokenizer vocab size: 32002
LLM Model vocab size: 32002
411993
432199488296135611


46129143145180560235159





OSError: Caught OSError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-21-0f926af0bbcc>", line 25, in __getitem__
    image = Image.open(image_path).convert("RGB")
  File "/usr/local/lib/python3.10/dist-packages/PIL/Image.py", line 3469, in open
    fp = builtins.open(filename, "rb")
OSError: [Errno 5] Input/output error: '/content/gdrive/MyDrive/LLMS_LLAVA/images/GCC_train_002059573.jpg'


In [ ]:

output_dir = './saved_weights'
os.makedirs(output_dir, exist_ok=True)

# Save the projection layer weights
torch.save(model.visual_projection.state_dict(), os.path.join(output_dir, 'visual_projection.pth'))

print("Projection weights saved successfully.")